In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import errno
import cvlib
import io


%matplotlib inline
#plt.style.use('dark_background') 
PATH = 'C:/GitHub/pythonPrograms/computerVision2023/proyectos/LicencePlates/'